In [52]:
import sys
sys.path.append("../code/")
from sklearn.metrics import mean_absolute_error
from dataloader import APPLIANCE_ORDER, get_train_test
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable

cuda_av = False
if torch.cuda.is_available():
    cuda_av = True

torch.manual_seed(0)
np.random.seed(0)


weight_appliance = {'mw':1, 'dw':1, 'dr':1,'fridge':1, 'hvac':1}

# num_hidden, num_iterations, num_layers, p, num_directions = sys.argv[1:6]

In [53]:
appliance="hvac"
cell_type = "GRU"
num_hidden = 120
num_iterations = 100
num_layers = 1
num_directions = 1

input_dim = 1
hidden_size = num_hidden
num_layers = num_layers
if num_directions == 1:
    bidirectional = False
else:
    bidirectional = True
lr = 0.1
p = 0.5
num_folds = 5
fold_num = 0

torch.manual_seed(0)

train, test = get_train_test(2, num_folds=num_folds, fold_num=fold_num)
train_aggregate = train[:, 0, None, :, :] - train[:, 1, None, :, :]
test_aggregate = test[:, 0, None, :, :]- test[:, 1, None, :, :]

In [75]:
t = Variable(torch.Tensor(train[:, 0, None, :, :]), requires_grad=True)
t.size()

torch.Size([54, 1, 112, 24])

In [76]:
t = t.view(-1, 1, 24)
t

Variable containing:
( 0  ,.,.) = 
   1366.9166   1367.6000    899.0000  ...    5006.9668   4656.8667   1456.9333

( 1  ,.,.) = 
    946.7667    891.6500    890.1833  ...    4071.2166   3348.8833    915.8834

( 2  ,.,.) = 
   1059.1000    788.1000    610.6000  ...    2488.2334   1840.3000   1359.0167
 ... 

(6045,.,.) = 
    579.5333    661.4833    606.6166  ...     948.5667    897.8333    987.7833

(6046,.,.) = 
   1062.3667    597.3666    719.2833  ...    1208.1000   1270.4333    810.2000

(6047,.,.) = 
    511.6000    480.5333    478.1833  ...    2177.7334   1905.5834   1545.0834
[torch.FloatTensor of size 6048x1x24]

In [77]:
q1 = nn.Conv1d(1, 30, 10, 1)
q2 = nn.Conv1d(30, 30, 8, 1)
q3 = nn.Conv1d(30, 40, 6, 1)
q4 = nn.Conv1d(40, 50, 3, 1)





q4(q3(q2(q1(t)))).size()

torch.Size([6048, 50, 1])

In [78]:
train_appliance = train[:, APPLIANCE_ORDER.index(appliance),None, :, :]
test_appliance = test[:, APPLIANCE_ORDER.index(appliance),None, :, :]

In [112]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        torch.manual_seed(0)

      
        self.q1 = nn.Conv1d(1, 30, 10, 1)
        self.act1 = nn.ReLU()
        #self.bn1 = nn.BatchNorm1d(30)
        self.q2 = nn.Conv1d(30, 30, 8, 1)
        self.q3 = nn.Conv1d(30, 40, 6, 1)
        self.q4 = nn.Conv1d(40, 50, 3, 1)
        self.l = nn.Linear(50, 24)
        self.act2 = nn.ReLU()
    
        
        
    def forward(self, x):
        
        pred = self.q1(x)
        pred = self.act1(pred)
        pred = self.q2(pred)
        pred = self.q3(pred)
        pred = self.q4(pred).view(-1, 50)
        pred = self.l(pred)
        pred = self.act2(pred)
        
        return pred

In [113]:
c = CustomCNN()
c(t)
loss_func(c(t), out)

Variable containing:
 866.7244
[torch.FloatTensor of size 1]

In [114]:
loss_func = nn.L1Loss()
c = CustomCNN()
lr = 0.001
if cuda_av:
    c = c.cuda()
    loss_func = loss_func.cuda()
optimizer = torch.optim.Adam(c.parameters(), lr=lr)

In [115]:
inp = t
for it in range(num_iterations):
    out = Variable(torch.Tensor(train_appliance)).view(-1, 1, 24)
    if cuda_av:
        inp = inp.cuda()
        out = out.cuda()

    
    pred = c(inp)

    optimizer.zero_grad()
    

    
    loss = loss_func(pred, out)
    if it % 1 == 0:
        print(it, loss.data[0])

    loss.backward()
    optimizer.step()

0 866.7244262695312
1 804.1245727539062
2 743.2656860351562
3 696.125
4 664.680908203125
5 634.1055297851562
6 598.0985717773438
7 569.9586791992188
8 564.5447387695312
9 567.1678466796875
10 568.4572143554688
11 566.102783203125
12 562.0824584960938
13 559.0032958984375
14 556.5321044921875
15 553.56982421875
16 550.5823364257812
17 548.3372192382812
18 546.3447875976562
19 543.9716796875
20 541.8792724609375
21 540.7139892578125
22 539.7012329101562
23 537.9024658203125
24 535.7562866210938
25 533.8154907226562
26 531.8809814453125
27 529.7385864257812
28 527.898193359375
29 526.3650512695312
30 524.6600952148438
31 522.905029296875
32 521.371337890625
33 519.7022094726562
34 517.7832641601562
35 516.0220336914062
36 514.3650512695312
37 512.632080078125
38 510.96484375
39 509.14990234375
40 506.99603271484375
41 504.7365417480469
42 502.1293029785156
43 498.724853515625
44 494.1765441894531
45 488.4815368652344
46 483.189697265625
47 479.39666748046875
48 476.5201721191406
49 474.43

In [116]:
train_appliance.shape

(54, 1, 112, 24)

In [117]:
pred[0]

Variable containing:
  730.4091
  533.6808
  526.0863
  583.0676
  350.6947
  164.0210
   37.6288
   44.1697
   61.3470
   47.8102
   92.6445
    0.0000
    0.0000
    0.0000
    0.0000
   72.3168
  511.2070
 1801.6776
 1439.4352
 1410.1440
 1658.2799
 1545.6464
 1187.9563
  813.7744
[torch.FloatTensor of size 24]

In [118]:
pd.Series(pred[0].data.numpy()[0, :]).plot()
pd.Series(test_appliance[0,  :]).plot()

IndexError: too many indices for array

In [123]:
test_inp = Variable(torch.Tensor(test_aggregate).view(-1,1,24), requires_grad=False)
test_out = Variable(torch.Tensor(test_appliance).view(-1,1,24), requires_grad=False)


if cuda_av:
    test_inp = test_inp.cuda()
    test_out = test_out.cuda()
    
    

In [124]:
pd.Series(c(test_inp)[0].data.numpy()[0, 0, :]).plot()
pd.Series(test_out.data.numpy()[0, 0, 0, :]).plot()

IndexError: too many indices for array

In [125]:
p = c(test_inp)
p[p<0.] = 0.

In [126]:
loss_func(p, test_out)

Variable containing:
 529.2985
[torch.FloatTensor of size 1]

In [127]:
sns.heatmap(pd.DataFrame(list(c.parameters())[0].data.numpy().reshape(50, 2)))

ValueError: cannot reshape array of size 300 into shape (50,2)

In [ ]:
params = [test_inp, -2]
for i in range(len(ORDER)):
    params.append(None)
pr = a(*params)
pr = torch.clamp(pr, min=0.)
test_pred = torch.split(pr, test_aggregate.shape[0])
prediction_fold = [None for x in range(len(ORDER))]

if cuda_av:
    for appliance_num, appliance in enumerate(ORDER):
        prediction_fold[appliance_num] = test_pred[appliance_num].cpu().data.numpy().reshape(-1, 24)
else:
    for appliance_num, appliance in enumerate(ORDER):
        prediction_fold[appliance_num] = test_pred[appliance_num].data.numpy().reshape(-1, 24)
gt_fold = [None for x in range(len(ORDER))]
for appliance_num, appliance in enumerate(ORDER):
    gt_fold[appliance_num] = test[:, APPLIANCE_ORDER.index(appliance), :, :].reshape(test_aggregate.shape[0], -1,
                                                                                         1).reshape(-1, 24)


print([x.mean() for x in pred_split])
error = pd.Series({appliance:mean_absolute_error(gt_fold[appliance_num], prediction_fold[appliance_num]) for appliance_num, appliance in enumerate(ORDER)})